In [ ]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import pandas as pd
# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
from tqdm import tqdm
# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "end_to_end_project"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
def reduce(df):
	int_list = ['int', 'int32', 'int16']
	float_list = ['float', 'float32']
	for col in tqdm(df.columns):
		col_type = df[col].dtypes
		if col_type in int_list:
			c_min = df[col].min()
			c_max = df[col].max()
			if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
				df[col] = df[col].astype(np.int8)
			elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
				df[col] = df[col].astype(np.int16)
			elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
				df[col] = df[col].astype(np.int32)
		elif col_type in float_list:
			c_min = df[col].min()
			c_max = df[col].max()
			if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
				df[col] = df[col].astype(np.float16)
			elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
				df[col] = df[col].astype(np.float32)
	return df
def reduce_s(df):
	int_list = ['int', 'int32', 'int16']
	float_list = ['float', 'float32']
	col_type = df.dtypes
	if col_type in int_list:
		c_min = df.min()
		c_max = df.max()
		if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
			df = df.astype(np.int8)
		elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
			df = df.astype(np.int16)
		elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
			df = df.astype(np.int32)
	elif col_type in float_list:
		c_min = df.min()
		c_max = df.max()
		if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
			df = df.astype(np.float16)
		elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
			df = df.astype(np.float32)
	return df


In [ ]:
%%time
train_ads, train_feeds = pd.read_csv('d:/Data/2022_3_data/train/train_data_ads.csv'), pd.read_csv('d:/Data/2022_3_data/train/train_data_feeds.csv')
test_ads, test_feeds = pd.read_csv('d:/Data/2022_3_data/test/test_data_ads.csv'), pd.read_csv('d:/Data/2022_3_data/test/test_data_feeds.csv')
train_ads.shape, train_feeds.shape, test_ads.shape, test_feeds.shape

In [ ]:
train_ads = reduce(train_ads)
train_feeds = reduce(train_feeds)
test_ads = reduce(test_ads)
test_feeds = reduce(test_feeds)
train_ads.shape, train_feeds.shape, test_ads.shape, test_feeds.shape

In [ ]:
# 保存为pkl格式方便读取
train_ads.to_pickle('d:/Data/2022_3_data/train/train_data_ads.pkl')
train_feeds.to_pickle('d:/Data/2022_3_data/train/train_data_feeds.pkl')

test_ads.to_pickle('d:/Data/2022_3_data/test/test_data_ads.pkl')
test_feeds.to_pickle('d:/Data/2022_3_data/test/test_data_feeds.pkl')

In [ ]:
%%time
train_ads, train_feeds = pd.read_pickle('d:/Data/2022_3_data/train/train_data_ads.pkl'), pd.read_pickle('d:/Data/2022_3_data/train/train_data_feeds.pkl')
test_ads, test_feeds = pd.read_pickle('d:/Data/2022_3_data/test/test_data_ads.pkl'), pd.read_pickle('d:/Data/2022_3_data/test/test_data_feeds.pkl')
train_ads.shape, train_feeds.shape, test_ads.shape, test_feeds.shape
full_data=pd.concat([train_ads,test_ads],ignore_index=True)

In [ ]:
import pandas as pd
train_ads = pd.read_pickle('d:/Data/2022_3_data/train/train_data_ads.pkl')

In [ ]:
test_feeds.columns

In [ ]:
train_ads.describe()

In [ ]:
import seaborn  as sns
ff=[]
def field_plot():
    for i in train_ads.columns:
        if train_ads[i].nunique() < 20:
            ff.append(i)
    for f in ff:
        print(f'Distribution of {f} :', train_ads[f].nunique())
        print(train_ads[f].value_counts()/len(train_ads),'\n')
        plt.figure(figsize=(15,10))
        sns.countplot(x=f,hue='label',data=train_ads)

field_plot()   

In [ ]:
train_ads["time_YMD"]=train_ads["pt_d"]//10000-20220603
train_ads["hour"]=train_ads["pt_d"]%10000//100

In [ ]:
# plt.figure(figsize=(15,10))
# sns.countplot(x="hour",hue='label',data=train_ads)

In [ ]:
# train_ads.columns

In [ ]:
train_ads

In [ ]:
corr=train_ads.corr()

In [ ]:
corr["label"].sort_values(ascending=True)

In [ ]:
def adjust(df, key, feature):
	if key == 'uid':
		mean7 = df[df['pt_d'] < 8][feature].mean()
		std7 = df[df['pt_d'] < 8][feature].std()
		mean8 = df[(df['pt_d'] >= 8) & (df['coldu'] == 1)][feature].mean()
		std8 = df[(df['pt_d'] >= 8) & (df['coldu'] == 1)][feature].std()
		df.loc[(df['pt_d'] >= 8) & (df['coldu'] == 1), feature]= ((df[(df['pt_d'] >= 8) & (df['coldu'] == 1)][feature] - mean8) / std8 * std7 + mean7)
	return df

In [ ]:
# train_ads["user_id"].value_counts()

In [ ]:
# columns=['user_id','task_id','ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003', 'ad_close_list_v001', 'ad_close_list_v002', 'ad_close_list_v003','time_YMD']
# train_ads[columns][train_ads.user_id==200204].head(80)

In [ ]:
# test_ads

In [ ]:
# user_cat=train_ads.user_id.unique().tolist()
# test_ads["new_man"]=[1 if i in user_cat  else 0 for i in test_ads.user_id ]

In [ ]:
# len(set(test_ads[test_ads["new_man"]==0]["user_id"]))

***数据分析***

In [ ]:
train_ads.columns

In [ ]:
train_ads.cat

In [ ]:
train_ads["time_YMD"].value_counts()

In [ ]:
train_ads["user_id"].nunique()

In [ ]:
cold_num=[]

for i in range(1,8):
    uid_list=train_ads[train_ads["time_YMD"]<i-1]["user_id"].unique()
    cold_list=[]
    old_list=[]
    k=0
    for j  in train_ads[train_ads["time_YMD"]==i]["user_id"].unique():
        if j not in uid_list:
            k+=1
            cold_list.append(j)
        else:
            old_list.append(j)
    cold_num.append(k)

In [ ]:
cold_num

In [ ]:
df=train_ads[train_ads["time_YMD"]==1].iloc[:,:3]

In [ ]:
df

In [ ]:
i=0
j=0
k=0
a=0
b=0
c=0

In [ ]:
for t1,t2 in tqdm(df.iterrows()):
    if t2[2] in cold_list:     
        i+=1
        if t2[1]==0:
            j+=1
        else:
            k+=1
    else:
        a+=1
        if t2[1]==0:
            b+=1
        else:
            c+=1


In [ ]:
i,j,k,a,b,c

In [ ]:
i/a

In [ ]:
k/j

In [ ]:
c/b

In [ ]:
k/j/(c/b)

In [ ]:
cold_num

In [ ]:
cold_num

In [ ]:
sum(cold_num)

In [ ]:
full_data=pd.concat([train_ads,test_ads],ignore_index=True)

In [ ]:
full_data.columns

In [ ]:
from pandas.plotting import scatter_matrix
columns=['label', 'age', 'gender', 'residence', 'city',
       'city_rank', 'series_dev', 'series_group', 'emui_dev', 'device_name',
       'device_size', 'net_type','adv_prim_id', 'inter_type_cd', 'slot_id', 'site_id', 'spread_app_id',
       'hispace_app_tags', 'app_second_class', 'app_score', 'u_newsCatInterestsST', 'u_refreshTimes', 'u_feedLifeCycle',
       'time_YMD', 'hour']
scatter_matrix(full_data[columns])

**特征选择**

In [ ]:
numeric_X = train_ads.select_dtypes(include=['int', 'float'])
cate_cols=numeric_X.columns.tolist()
cate_cols

In [ ]:
def make_count_feather(df):

    print('开始构造count特征')
    cate_cols =['log_id','label','user_id', 'age','gender','residence','city','city_rank','series_dev','series_group','emui_dev','device_name','device_size','net_type','task_id','adv_id',
 'creat_type_cd',
 'adv_prim_id',
 'inter_type_cd',
 'slot_id',
 'site_id',
 'spread_app_id',
 'hispace_app_tags',
 'app_second_class',
 'pt_d',
 'u_refreshTimes',
 'u_feedLifeCycle',
 'time_YMD',
 'hour']
    for f in tqdm(cate_cols):
        tmp = df[f].map(df[f].value_counts())
        if tmp.var() > 1:
            df[f + '_count'] = tmp
           #df = adjust_single(df, f, f + '_count')
    return df
     

In [ ]:
make_count_feather(train_ads)

In [ ]:
train_ads.columns

In [ ]:
# cate_cols

In [ ]:
def group_fea(df,key,target):
	tmp = df.groupby(key, as_index=False)[target].agg({
		key + '_' + target + '_nunique': 'nunique',
	}).reset_index().drop('index', axis=1)
	return tmp

In [ ]:
# nunique特征
print('开始构造nunique特征')
nunique_group = []

print('用户')
key = 'user_id'
feature_target = ['task_id', 'adv_id', 'creat_type_cd','inter_type_cd', 'slot_id',  'site_id', 'spread_app_id','Tags','app_second_class','app_score']
for target in tqdm(feature_target):
    if key + '_' + target + '_nunique' not in nunique_group:
        nunique_group.append(key + '_' + target + '_nunique')
        tmp = group_fea(df,key,target)
        df = df.merge(tmp,on=key,how='left')
       # df = adjust_single(df, key, key + '_' + target + '_nunique')
    if target + '_' + key + '_nunique' not in nunique_group:
        nunique_group.append(target + '_' + key + '_nunique')
        tmp = group_fea(df,target,key)
        df = df.merge(tmp,on=target,how='left')

print('广告')
key = 'adv_id'
feature_target = ['user_id', 'age', 'city', 'city_rank','device_name', 'device_size', 'net_type','emui_dev', 'residence', 'gender', 'adv_prim_id','site_id', 'slot_id', 'spread_app_id']
for target in tqdm(feature_target):
    if key + '_' + target + '_nunique' not in nunique_group:
        nunique_group.append(key + '_' + target + '_nunique')
        tmp = group_fea(df,key,target)
        df = df.merge(tmp,on=key,how='left')
    if target + '_' + key + '_nunique' not in nunique_group:
        nunique_group.append(target + '_' + key + '_nunique')
        tmp = group_fea(df,target,key)
        df = df.merge(tmp,on=target,how='left')

print('清除')
for feature in tqdm(nunique_group):
    if df[feature].var()<1:
        df = df.drop(feature, axis=1)

df = reduce(df)


In [ ]:
from gensim.models import Word2Vec
import networkx as nx

In [ ]:
test_df=train_ads

In [ ]:
emb_cols = [['user_id', 'adv_id']]
sort_df = test_df.sort_values('pt_d').reset_index(drop=True)


In [ ]:
for f1, f2 in emb_cols:

    tmp, tmp2 = emb_adjust(sort_df, f1, f2)
    # df = df.merge(tmp, on=f1, how='left').merge(tmp2, on=f2, how='left').fillna(0)

In [ ]:
def emb_adjust(df, f1, f2):
	emb_size = 8
	df = df.fillna(0)
	tmp = df.groupby(f1, as_index=False)[f2].agg({'{}_{}_list'.format(f1, f2): list})
	sentences = tmp['{}_{}_list'.format(f1, f2)].values.tolist()
	for i in range(len(sentences)):
		sentences[i] = [str(x) for x in sentences[i]]
	model = Word2Vec(sentences, vector_size=emb_size, window=6, min_count=5, sg=0, hs=0, seed=1, epochs=5)

	index_dict = {}
	emb_matrix = []
	for i in tqdm(range(len(sentences))):
		seq = sentences[i]
		vec = []
		for w in seq:
			if w in model.wv.index_to_key:
				#print(model.wv.index_to_key)
				vec.append(model.wv[w])
				#print(vec)
		if len(vec) > 0:
			emb_matrix.append(np.mean(vec, axis=0))
		else:
			emb_matrix.append([0] * emb_size)
		index_dict[tmp[f1][i]] = i
	emb_matrix = np.array(emb_matrix)
	for i in range(emb_size):
		tmp['{}_of_{}_emb_{}'.format(f1, f2, i)] = emb_matrix[:, i]

	tmp_f2 = df.groupby(f2, as_index=False)[f1].agg({'{}_{}_list'.format(f2, f1): list})
	sentences_f2 = tmp_f2['{}_{}_list'.format(f2, f1)].values.tolist()
	index_dict_f2 = {}
	emb_matrix_f2 = []
	for i in tqdm(range(len(sentences_f2))):
		seq = sentences_f2[i]
		vec = []
		for w in seq:
			vec.append(emb_matrix[index_dict[w]])
		if len(vec) > 0:
			emb_matrix_f2.append(np.mean(vec, axis=0))
		else:
			emb_matrix_f2.append([0] * emb_size)
		index_dict_f2[str(tmp_f2[f2][i])] = i
	emb_matrix_f2 = np.array(emb_matrix_f2)

	emb_matrix_adjust = []
	for seq in tqdm(sentences):
		vec = []
		for w in seq:
			vec.append(emb_matrix_f2[index_dict_f2[w]])
		if len(vec) > 0:
			emb_matrix_adjust.append(np.mean(vec, axis=0))
		else:
			emb_matrix_adjust.append([0] * emb_size)
	emb_matrix_adjust = np.array(emb_matrix_adjust)
	for i in range(emb_size):
		tmp['{}_of_{}_emb_adjust_{}'.format(f1, f2, i)] = emb_matrix_adjust[:, i]

	tmp = tmp.drop('{}_{}_list'.format(f1, f2), axis=1)
	
	word_list = []
	emb_matrix2 = []
	for w in tqdm(model.wv.index_to_key):
		word_list.append(w)
		emb_matrix2.append(model.wv[w])
	emb_matrix2 = np.array(emb_matrix2)
	tmp2 = pd.DataFrame()
	tmp2[f2] = np.array(word_list).astype('int')
	for i in range(emb_size):
		tmp2['{}_emb_{}'.format(f2, i)] = emb_matrix2[:, i]
	
	return tmp, tmp2

In [ ]:
df=full_data

In [ ]:
# embedding特征
print('开始构造emb特征')
emb_cols = [['user_id', 'adv_id']]
sort_df = df.sort_values('pt_d').reset_index(drop=True)
for f1, f2 in emb_cols:
    tmp, tmp2 = emb_adjust(sort_df, f1, f2)
    #df = df.merge(tmp, on=f1, how='left').merge(tmp2, on=f2, how='left').fillna(0)

In [ ]:
tmp

In [ ]:
tmp2

In [ ]:
df=train_ads[:500]

In [ ]:
# ctr特征
print('开始构造ctr特征')
mean_rate = df[df['time_YMD'] < 2]['label'].mean()
feature_list = cate_cols
for feat_1 in tqdm(feature_list):
    res = pd.DataFrame()
    for period in [0, 1, 2, 3, 4, 5, 6, 7]:  #注意在实际训练过程中
        if period == 1:
            count = df[df['time_YMD'] <= period].groupby(feat_1, as_index=False)['label'].agg({feat_1 + '_rate': 'mean'})
        else:
            count = df[df['time_YMD'] < period].groupby(feat_1, as_index=False)['label'].agg({feat_1 + '_rate': 'mean'})
        count['time_YMD'] = period
        res = res.append(count, ignore_index=True)
    df = pd.merge(df, res, how='left', on=[feat_1, 'time_YMD'], sort=False)
    #df[feat_1 + '_rate'] = reduce_s(df[feat_1 + '_rate'].fillna(mean_rate))
    #print(feat_1, ' over')


In [ ]:
def adjust_single(df, key, feature):
	if key == 'uid':
		mean7 = df[df['pt_d'] < 8].drop_duplicates(['uid'])[feature].mean()
		std7 = df[df['pt_d'] < 8].drop_duplicates(['uid'])[feature].std()
		mean8 = df[(df['pt_d'] >= 8) & (df['coldu'] == 1)].drop_duplicates(['uid'])[feature].mean()
		std8 = df[(df['pt_d'] >= 8) & (df['coldu'] == 1)].drop_duplicates(['uid'])[feature].std()
		df.loc[(df['pt_d'] == 10) & (df['coldu'] == 1) & (df['coldt'] == 0), feature]= ((df[(df['pt_d'] == 10) & (df['coldu'] == 1) & (df['coldt'] == 0)][feature] - mean8) / std8 * std7 + mean7 * 1.1)
		df.loc[(df['pt_d'] == 10) & (df['coldu'] == 1) & (df['coldt'] == 1), feature]= ((df[(df['pt_d'] == 10) & (df['coldu'] == 1) & (df['coldt'] == 1)][feature] - mean8) / std8 * std7 * 0.8 + mean7 * 0.8)
	return df

In [ ]:
def atom_prediction(i, fold, epoch=550):
	params = {
		'boosting_type': 'gbdt',
		'objective': 'binary',
		'metric': 'auc',
		'boost_from_average' : True,
		'train_metric': True, 
		'feature_fraction_seed' : 1,
		'learning_rate': 0.05,
		'is_unbalance': False,  #当训练数据是不平衡的，正负样本相差悬殊的时候，可以将这个属性设为true,此时会自动给少的样本赋予更高的权重
		'num_leaves': 256,  # 一般设为少于2^(max_depth)
		'max_depth': -1,  #最大的树深，设为-1时表示不限制树的深度
		'min_child_samples': 15,  # 每个叶子结点最少包含的样本数量，用于正则化，避免过拟合
		'max_bin': 200,  # 设置连续特征或大量类型的离散特征的bins的数量
		'subsample': 1,  # Subsample ratio of the training instance.
		'subsample_freq': 1,  # frequence of subsample, <=0 means no enable
		'colsample_bytree': 0.5,  # Subsample ratio of columns when constructing each tree.
		'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
		'subsample_for_bin': 200000,  # Number of samples for constructing bin
		'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
		'reg_alpha': 2.99,  # L1 regularization term on weights
		'reg_lambda': 1.9,  # L2 regularization term on weights
		'nthread': 20,
		'verbose': 0,
		}
	
	print('predict fold:{}'.format(i + 1))
	df = pd.read_pickle('./data/feature/fea_'+str(fold)+'_'+str(i+1)+'.pkl')
	
	test_df = df[df["pt_d"]==10].reset_index(drop=True)
	train_df = df[df["pt_d"]<8].reset_index(drop=True)

	X_train = train_df
	y_train = X_train["label"].astype('int32')
	drop_fea = ['index', 'id', 'pt_d', 'coldu', 'label', 'communication_onlinerate', 'testb', 'coldt']
	feature = [x for x in X_train.columns if x not in drop_fea]
	print(feature)
	weight = X_train['pt_d'] / X_train['pt_d'].max()
	lgb_train = lgb.Dataset(X_train[feature], y_train, weight = weight)

	gbm = lgb.train(params, lgb_train, num_boost_round=epoch,  valid_sets=(lgb_train), verbose_eval = 50)
	# gbm = lgb.train(params, lgb_train, num_boost_round=epoch)

	preds = gbm.predict(test_df[feature], num_iteration=gbm.best_iteration) / fold

	res = pd.DataFrame()
	res['id'] = test_df['id'].astype('int32')
	res['probability'] = preds
	res['probability'] = res['probability'].astype(np.float32)
	res.to_csv('result/submission_'+str(fold)+'_'+str(i+1)+'.csv',index = False)

	print('save fold:{}'.format(i + 1))
	return

from joblib import Parallel, delayed
if __name__ == "__main__":

#     logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s', level=logging.INFO)

	df_raw = pd.read_pickle('./data/train_data.pkl')

	df0 = df_raw[df_raw['label']==0].copy()
	df1 = df_raw[df_raw['label']==1].copy()

	test_df_A  = pd.read_pickle('./data/test_data_A.pkl')
	test_df_B = pd.read_pickle('./data/test_data_B.pkl')
	# test_df_A['pt_d'] = 10
	# test_df_A['testb'] = 0
	# test_df_B['testb'] = 1
	train_uid = set(df_raw['uid'])
	test_df_A['coldu'] = test_df_A['uid'].apply(lambda x: 1 if x not in train_uid else 0)
	test_df_B['coldu'] = test_df_B['uid'].apply(lambda x: 1 if x not in train_uid else 0)
	train_uid = set(list(set(df_raw['uid'])) + list(set(test_df_A['uid'])))
	test_df_B['coldt'] = test_df_B['uid'].apply(lambda x: 1 if x not in train_uid else 0)
	test_df_raw = pd.concat([test_df_A,test_df_B], axis=0).reset_index(drop=True)
	# train_uid = set(df_raw['uid'])
	# test_df_raw['coldu'] = test_df_raw['uid'].apply(lambda x: 1 if x not in train_uid else 0)
	del df_raw

	epoch = 550
	fold = 4
	preds = 0
	print('开始{}折制作特征'.format(fold))
	skf = StratifiedKFold(n_splits=fold, shuffle=True, random_state=1)
	Parallel(n_jobs=fold)(delayed(atom_makefea)(i,trn_idx, val_idx, df0, df1,test_df_raw, fold) for i, (trn_idx, val_idx) in enumerate(skf.split(df0, df0['pt_d'])))

	print('开始{}折训练'.format(fold))
	# Parallel(n_jobs=2)(delayed(atom_prediction)(i, fold, epoch) for i in range(fold))
	for i in range(fold):
		atom_prediction(i, fold, epoch)
	
	print('开始{}折结果融合'.format(fold))
	preds = np.zeros(2000000)
	res = pd.read_csv('result/submission_'+str(fold)+'_'+str(1)+'.csv')
	for i in range(fold):
		res_ = pd.read_csv('result/submission_'+str(fold)+'_'+str(i+1)+'.csv')
		preds+=res_['probability']
	res['probability'] = preds
	res['probability'] = res['probability'].astype(np.float32)
	res.to_csv('result/submission_'+str(fold)+'_all.csv',index = False)

**目标编码**

In [ ]:
import os
import pandas as pd
import numpy as np
import random
import json
import gc
from gensim.corpora import WikiCorpus
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from collections import Counter
from sklearn import preprocessing
import scipy.special as special
from pandas import DataFrame, Series
from tqdm import tqdm
import time
from sklearn.model_selection import KFold
np.random.seed(2019)
random.seed(2019)


In [ ]:
train_ads.columns

In [ ]:
one=train_ads

In [ ]:
one

In [ ]:
ff=[]
for i in train_ads.columns:
        if train_ads[i].nunique() > 12:
            ff.append(i)
for f in ff:
        print(f'Distribution of {f} :', train_ads[f].nunique())
        print(train_ads[f].value_counts()/len(train_ads),'\n')

In [ ]:
columns=["residence","city","series_dev","emui_dev","device_name","device_size","hispace_app_tags","app_second_class"]

In [ ]:
train_ads_sample=train_ads
train_ads,test_ads=train_ads_sample[:500],train_ads_sample[500:600]

In [ ]:
columns=["residence","city","series_dev","emui_dev","device_name","device_size","hispace_app_tags","app_second_class"]
target_col="label"
test=test_ads
# N_min -> smoothing term, minimum sample size, if sample size is less than N_min, add up to N_min 
N_min=20
kf = KFold(n_splits=10, shuffle=True, random_state=0)
for i, (dev_index, val_index) in enumerate(kf.split(train_ads.index.values)):
    # split data into dev set and validation set
    dev = train_ads.loc[dev_index].reset_index(drop=True) 
    val = train_ads.loc[val_index].reset_index(drop=True)
        
    feature_cols = []    
    for var_name in columns:
        feature_name = f'{var_name}_mean'
        feature_cols.append(feature_name)
        
        prior_mean = np.mean(dev[target_col])
        stats = dev[[target_col, var_name]].groupby(var_name).agg(['sum', 'count'])[target_col].reset_index()           
   
        ### beta target encoding by Bayesian average for dev set 
        df_stats = pd.merge(dev[[var_name]], stats, how='left')
        df_stats['sum'].fillna(value = prior_mean, inplace = True)
        df_stats['count'].fillna(value = 1.0, inplace = True)
        N_prior = np.maximum(N_min - df_stats['count'].values, 0)   # prior parameters
        dev[feature_name] = (prior_mean * N_prior + df_stats['sum']) / (N_prior + df_stats['count']) # Bayesian mean
 
        ### beta target encoding by Bayesian average for val set
        df_stats = pd.merge(val[[var_name]], stats, how='left')
        df_stats['sum'].fillna(value = prior_mean, inplace = True)
        df_stats['count'].fillna(value = 1.0, inplace = True)
        N_prior = np.maximum(N_min - df_stats['count'].values, 0)   # prior parameters
        val[feature_name] = (prior_mean * N_prior + df_stats['sum']) / (N_prior + df_stats['count']) # Bayesian mean
        
        ### beta target encoding by Bayesian average for test set
        df_stats = pd.merge(test[[var_name]], stats, how='left')
        df_stats['sum'].fillna(value = prior_mean, inplace = True)
        df_stats['count'].fillna(value = 1.0, inplace = True)
        N_prior = np.maximum(N_min - df_stats['count'].values, 0)   # prior parameters
        test.loc[:,feature_name] = (prior_mean * N_prior + df_stats['sum']) / (N_prior + df_stats['count']) # Bayesian mean
        
        # Bayesian mean is equivalent to adding N_prior data points of value prior_mean to the data set.
        del df_stats, stats
    # Step 3: train model (K-fold CV), get oof prediction

In [ ]:
dev

In [ ]:
val

In [ ]:
test

***lightgbm的设计方法***

In [ ]:
import lightgbm as lgb

In [ ]:
def atom_prediction(i, fold, epoch=550):
	params = {
		'boosting_type': 'gbdt',
		'objective': 'binary',
		'metric': 'auc',
		'boost_from_average' : True,
		'train_metric': True, 
		'feature_fraction_seed' : 1,
		'learning_rate': 0.05,
		'is_unbalance': False,  #当训练数据是不平衡的，正负样本相差悬殊的时候，可以将这个属性设为true,此时会自动给少的样本赋予更高的权重
		'num_leaves': 256,  # 一般设为少于2^(max_depth)
		'max_depth': -1,  #最大的树深，设为-1时表示不限制树的深度
		'min_child_samples': 15,  # 每个叶子结点最少包含的样本数量，用于正则化，避免过拟合
		'max_bin': 200,  # 设置连续特征或大量类型的离散特征的bins的数量
		'subsample': 1,  # Subsample ratio of the training instance.
		'subsample_freq': 1,  # frequence of subsample, <=0 means no enable
		'colsample_bytree': 0.5,  # Subsample ratio of columns when constructing each tree.
		'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
		'subsample_for_bin': 200000,  # Number of samples for constructing bin
		'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
		'reg_alpha': 2.99,  # L1 regularization term on weights
		'reg_lambda': 1.9,  # L2 regularization term on weights
		'nthread': 20,
		'verbose': 0,
		}
	
	print('predict fold:{}'.format(i + 1))
	df = pd.read_pickle('./data/feature/fea_'+str(fold)+'_'+str(i+1)+'.pkl')
	
	test_df = df[df["pt_d"]==10].reset_index(drop=True)
	train_df = df[df["pt_d"]<8].reset_index(drop=True)

	X_train = train_df
	y_train = X_train["label"].astype('int32')
	drop_fea = ['index', 'id', 'pt_d', 'coldu', 'label', 'communication_onlinerate', 'testb', 'coldt']
	feature = [x for x in X_train.columns if x not in drop_fea]
	print(feature)
	weight = X_train['pt_d'] / X_train['pt_d'].max()
	lgb_train = lgb.Dataset(X_train[feature], y_train, weight = weight)

	gbm = lgb.train(params, lgb_train, num_boost_round=epoch,  valid_sets=(lgb_train), verbose_eval = 50)
	# gbm = lgb.train(params, lgb_train, num_boost_round=epoch)

	preds = gbm.predict(test_df[feature], num_iteration=gbm.best_iteration) / fold

	res = pd.DataFrame()
	res['id'] = test_df['id'].astype('int32')
	res['probability'] = preds
	res['probability'] = res['probability'].astype(np.float32)
	res.to_csv('result/submission_'+str(fold)+'_'+str(i+1)+'.csv',index = False)

	print('save fold:{}'.format(i + 1))
	return


In [ ]:
def atom_makefea(i,trn_idx, val_idx, df0, df1,test_df_raw, fold):
	print('fold:{}'.format(i + 1))
	df = pd.concat([df0.iloc[val_idx].reset_index(drop=True),df1], axis=0).reset_index(drop=True)

	df = pd.concat([df,test_df_raw], axis=0).reset_index(drop=True)

	#df = make_feature(df)

	df.to_pickle('./data/feature/fea_'+str(fold)+'_'+str(i+1)+'.pkl')
	print('save fea to fea.pkl!', i+1)
	return 


In [ ]:
from joblib import Parallel, delayed
if __name__ == "__main__":

#     logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s', level=logging.INFO)

	df_raw = pd.read_pickle('./data/train_data.pkl')

	df0 = df_raw[df_raw['label']==0].copy()
	df1 = df_raw[df_raw['label']==1].copy()

	test_df_A  = pd.read_pickle('./data/test_data_A.pkl')
	test_df_B = pd.read_pickle('./data/test_data_B.pkl')
	# test_df_A['pt_d'] = 10
	# test_df_A['testb'] = 0
	# test_df_B['testb'] = 1
	train_uid = set(df_raw['uid'])
	test_df_A['coldu'] = test_df_A['uid'].apply(lambda x: 1 if x not in train_uid else 0)
	test_df_B['coldu'] = test_df_B['uid'].apply(lambda x: 1 if x not in train_uid else 0)
	train_uid = set(list(set(df_raw['uid'])) + list(set(test_df_A['uid'])))
	test_df_B['coldt'] = test_df_B['uid'].apply(lambda x: 1 if x not in train_uid else 0)
	test_df_raw = pd.concat([test_df_A,test_df_B], axis=0).reset_index(drop=True)
	# train_uid = set(df_raw['uid'])
	# test_df_raw['coldu'] = test_df_raw['uid'].apply(lambda x: 1 if x not in train_uid else 0)
	del df_raw

	epoch = 550
	fold = 4
	preds = 0
	print('开始{}折制作特征'.format(fold))
	skf = StratifiedKFold(n_splits=fold, shuffle=True, random_state=1)
	Parallel(n_jobs=fold)(delayed(atom_makefea)(i,trn_idx, val_idx, df0, df1,test_df_raw, fold) for i, (trn_idx, val_idx) in enumerate(skf.split(df0, df0['pt_d'])))

	print('开始{}折训练'.format(fold))
	# Parallel(n_jobs=2)(delayed(atom_prediction)(i, fold, epoch) for i in range(fold))
	for i in range(fold):
		atom_prediction(i, fold, epoch)
	
	print('开始{}折结果融合'.format(fold))
	preds = np.zeros(2000000)
	res = pd.read_csv('result/submission_'+str(fold)+'_'+str(1)+'.csv')
	for i in range(fold):
		res_ = pd.read_csv('result/submission_'+str(fold)+'_'+str(i+1)+'.csv')
		preds+=res_['probability']
	res['probability'] = preds
	res['probability'] = res['probability'].astype(np.float32)
	res.to_csv('result/submission_'+str(fold)+'_all.csv',index = False)



#TODO  记得加入冷启动的属性！

***滑动窗口的设计与穿梭特征的构建***

In [ ]:
#现在不考虑实现label的平滑
#比例偏好
df=full_data

In [ ]:
def time_groupby(df, win, key, target):
    tmp = df.groupby([key, 'time_YMD'], as_index=False)[target].agg({
        key + '_' + target + '_win_nunique': 'nunique',
    }).reset_index().drop('index', axis=1)
    tmp = tmp.set_index([key, "time_YMD"])[[key + '_' + target + '_win_nunique']].unstack(level=-1).fillna(0)
    tmp.columns = tmp.columns.get_level_values(1)
    tmp = tmp.T
    tmp = tmp.rolling(window=win).sum().fillna(0)
    tmp = tmp.stack().reset_index()
    tmp.columns = ['time_YMD', key, key + '_' + target + '_win_nunique']

    tmp12 = tmp[tmp['time_YMD'] < win].copy()
    del tmp12[key + '_' + target + '_win_nunique']
    tmp3 = tmp[tmp['time_YMD'] == win].copy()
    del tmp3['time_YMD']
    tmp12 = tmp12.merge(tmp3, on=key, how='left')
    tmp7 = tmp[tmp['time_YMD'] > (win - 1)].copy()
    tmp = pd.concat([tmp12, tmp7])

    #     if tmp['win3_slide_'+fea+'sum'].var()>1:

    #     df=reduce(df)
    return tmp

In [ ]:
# 滑窗groupy
#需要注意time_groupby函数中win的调参，建议设置为5，也可以尝试3,4
nunique_group = []
key = 'user_id'
feature_target = ['task_id', 'adv_id', 'creat_type_cd','inter_type_cd','hispace_app_tags', 'adv_prim_id', 'slot_id', 'spread_app_id']
for target in tqdm(feature_target):
    if key + '_' + target + '_win_nunique' not in nunique_group:
        nunique_group.append(key + '_' + target + '_nunique')
        tmp = time_groupby(df, 5, key, target)
        df = df.merge(tmp, on=[key, 'time_YMD'], how='left')
        #df = adjust(df, key, key + '_' + target + '_win_nunique')
    if target + '_' + key + '_win_nunique' not in nunique_group:
        nunique_group.append(target + '_' + key + '_win_nunique')
        tmp = time_groupby(df, 5, target, key)
        df = df.merge(tmp, on=[target, 'time_YMD'], how='left')
        #df = adjust(df, target, target + '_' + key + '_win_nunique')

In [ ]:
#穿越特征的构建
# data.groupby(col)['date'].shift(-gap)
# data["ts_{}_{}_diff_next".format("_".join(col),gap)]-data["date"]

# data.groupby(col)['date'].shift(+gap)
# data['date']-data["ts_{}_{}_diff_next".format("_".join(col),gap)]

In [ ]:
full_data.columns

In [ ]:
#sample=full_data.sample(10000)

In [ ]:
full_data

In [ ]:
def throughfeather(full_data,gap,cols):
    full_data = full_data.sort_values('pt_d')
    for col in cols:
        full_data["ts_{}_{}_diff_next_date".format("_".join(col),gap)]=full_data.groupby(col)['pt_d'].transform(lambda x:x.shift(-gap))
        full_data["ts_{}_{}_diff_next".format("_".join(col),gap)]=minute(full_data["ts_{}_{}_diff_next_date".format("_".join(col),gap)])-minute(full_data["pt_d"])
        del full_data["ts_{}_{}_diff_next_date".format("_".join(col),gap)]
    return full_data


def minute(j):
        return ((j%1000000)//10000)*24*60+((j%100000000))//100*60+(j%100)


In [ ]:
col=["user_id","task_id","adv_id"]
full_data=throughfeather(train_ads,1,col)

In [ ]:
full_data[full_data["user_id"]==141810]

**鱼佬建议**

In [ ]:
#----------------环境配置----------------
#安装相关依赖库 如果是windows系统，cmd命令框中输入pip安装，参考上述环境配置
#!pip install sklearn
#!pip install pandas
#!pip install catboost
#--------------------------------------

#----------------导入库-----------------
# 数据探索模块使用第三方库
import pandas as pd
import numpy as np
import os
import gc
import matplotlib.pyplot as plt
from tqdm import *
# 核心模型使用第三方库
from catboost import CatBoostClassifier
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge
# 交叉验证所使用的第三方库
from sklearn.model_selection import StratifiedKFold, KFold
# 评估指标所使用的的第三方库
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
# 忽略报警所使用的第三方库
import warnings
warnings.filterwarnings('ignore')
#--------------------------------------

# ----------------数据预处理-------------
# 读取训练数据和测试数据
train_data_ads = pd.read_csv('./2022_3_data/train/train_data_ads.csv')
train_data_feeds = pd.read_csv('./2022_3_data/train/train_data_feeds.csv')

test_data_ads = pd.read_csv('./2022_3_data/test/test_data_ads.csv')
test_data_feeds = pd.read_csv('./2022_3_data/test/test_data_feeds.csv')

# 合并数据
train_data_ads['istest'] = 0
test_data_ads['istest'] = 1
data_ads = pd.concat([train_data_ads, test_data_ads], axis=0, ignore_index=True)

train_data_feeds['istest'] = 0
test_data_feeds['istest'] = 1
data_feeds = pd.concat([train_data_feeds, test_data_feeds], axis=0, ignore_index=True)

del train_data_ads, test_data_ads, train_data_feeds, test_data_feeds
gc.collect()

# ----------------特征工程---------------
# 包含自然数编码、特征提取和内存压缩三部分内容。
# 1、自然数编码
def label_encode(series, series2):
    unique = list(series.unique())
    return series2.map(dict(zip(
        unique, range(series.nunique())
    )))

for col in ['ad_click_list_v001','ad_click_list_v002','ad_click_list_v003','ad_close_list_v001','ad_close_list_v002','ad_close_list_v003','u_newsCatInterestsST']:
    data_ads[col] = label_encode(data_ads[col], data_ads[col])

# 2、特征提取
# data_feeds特征构建
# 特征提取部分围绕着data_feeds进行构建（添加源域信息）
# 主要是nunique属性数统计和mean均值统计。
# 由于是baseline方案，所以整体的提取比较粗暴，大家还是有很多的优化空间。
cols = [f for f in data_feeds.columns if f not in ['label','istest','u_userId']]
for col in tqdm(cols):
    tmp = data_feeds.groupby(['u_userId'])[col].nunique().reset_index()
    tmp.columns = ['user_id', col+'_feeds_nuni']
    data_ads = data_ads.merge(tmp, on='user_id', how='left')

cols = [f for f in data_feeds.columns if f not in ['istest','u_userId','u_newsCatInterests','u_newsCatDislike','u_newsCatInterestsST','u_click_ca2_news','i_docId','i_s_sourceId','i_entities']]
for col in tqdm(cols):
    tmp = data_feeds.groupby(['u_userId'])[col].mean().reset_index()
    tmp.columns = ['user_id', col+'_feeds_mean']
    data_ads = data_ads.merge(tmp, on='user_id', how='left')

# 3、内存压缩
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# 压缩使用内存
# 由于数据比较大，所以合理的压缩内存节省空间尤为的重要
# 使用reduce_mem_usage函数可以压缩近70%的内存占有。
data_ads = reduce_mem_usage(data_ads)
# Mem. usage decreased to 2351.47 Mb (69.3% reduction)
#--------------------------------------

# ----------------数据集划分-------------
# 划分训练集和测试集
cols = [f for f in data_ads.columns if f not in ['label','istest']]
x_train = data_ads[data_ads.istest==0][cols]
x_test = data_ads[data_ads.istest==1][cols]

y_train = data_ads[data_ads.istest==0]['label']

del data_ads, data_feeds
gc.collect()
#--------------------------------------

# ----------------训练模型---------------
def cv_model(clf, train_x, train_y, test_x, clf_name, seed=2022):
    
    kf = KFold(n_splits=5, shuffle=True, random_state=seed)

    train = np.zeros(train_x.shape[0])
    test = np.zeros(test_x.shape[0])

    cv_scores = []

    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print('************************************ {} {}************************************'.format(str(i+1), str(seed)))
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index], train_y[valid_index]
               
        params = {'learning_rate': 0.3, 'depth': 5, 'l2_leaf_reg': 10, 'bootstrap_type':'Bernoulli','random_seed':seed,
                  'od_type': 'Iter', 'od_wait': 50, 'random_seed': 11, 'allow_writing_files': False}

        model = clf(iterations=20000, **params, eval_metric='AUC')
        model.fit(trn_x, trn_y, eval_set=(val_x, val_y),
                  metric_period=200,
                  cat_features=[], 
                  use_best_model=True, 
                  verbose=1)

        val_pred  = model.predict_proba(val_x)[:,1]
        test_pred = model.predict_proba(test_x)[:,1]
            
        train[valid_index] = val_pred
        test += test_pred / kf.n_splits
        cv_scores.append(roc_auc_score(val_y, val_pred))
        
        print(cv_scores)
       
    print("%s_score_list:" % clf_name, cv_scores)
    print("%s_score_mean:" % clf_name, np.mean(cv_scores))
    print("%s_score_std:" % clf_name, np.std(cv_scores))
    return train, test

cat_train, cat_test = cv_model(CatBoostClassifier, x_train, y_train, x_test, "cat")
#--------------------------------------

# ----------------结果保存---------------
x_test['pctr'] = cat_test
x_test[['log_id','pctr']].to_csv('submission.csv', index=False)

**graph embedding**

In [ ]:
def randomWalk(_g, _corpus_num, _deep_num, _current_word):
	_corpus = []
	for _ in range(_corpus_num):
		sentence = [_current_word]
		current_word = _current_word
		count = 0
		while count<_deep_num:
			count+=1
			_node_list = list(_g[current_word].keys())
			_weight_list = np.array([item['weight'] for item in (_g[current_word].values())])
			_ps = _weight_list / np.sum(_weight_list)
			sel_node = roulette(_node_list, _ps)
			if count % 2 == 0:
				sentence.append(sel_node)
			current_word = sel_node
		_corpus.append(sentence)
	return _corpus

def roulette(_datas, _ps):
	return np.random.choice(_datas, p=_ps)

def build_graph(df, f1, f2):
	G = nx.Graph()
	df_weight = df.groupby([f1, f2], as_index=False)['gender'].agg({'weight': 'count',}).reset_index().drop('index', axis=1)
	df_weight[f1 + '_word'] = df_weight[f1].astype(str) + '_' + f1
	df_weight[f2 + '_word'] = df_weight[f2].astype(str) + '_' + f2
	df_weight = df_weight.drop(f1, axis=1).drop(f2, axis=1)
	for i in tqdm(range(len(df_weight))):
		G.add_edge(df_weight[f1 + '_word'][i], df_weight[f2 + '_word'][i], weight=df_weight['weight'][i])
	return G, df_weight

def deep_walk(G, df_weight, f1, f2):
	num = 5
	deep_num = 20
	f2_set = set(df_weight[f2 + '_word'])
	sentences = []
	for word in tqdm(f2_set):
		corpus = randomWalk(G, num, deep_num, word)
		sentences += corpus
	return sentences

def deep_walk_pool(G, f2_set, f1, f2):
	num = 5
	deep_num = 40
	sentences = []
	for word in tqdm(f2_set):
		corpus = randomWalk(G, num, deep_num, word)
		sentences += corpus
	return sentences

def graph_emb(sentences, G, df_weight, f1, f2):
	emb_size = 8
	model = Word2Vec(sentences, size=emb_size, window=6, min_count=5, sg=0, hs=0, seed=1, iter=5)
	f1_set = list(set(df_weight[f1 + '_word']))
	emb_matrix = []
	tmp = pd.DataFrame()
	tmp[f1 + '_word'] = f1_set
	for f1_word in f1_set:
		vec = []
		for f2_word in G[f1_word].keys():
			if f2_word in model.wv.vocab:
				vec.append(model.wv[f2_word])
		if len(vec) > 0:
			emb_matrix.append(np.mean(vec, axis=0))
		else:
			emb_matrix.append([0] * emb_size)
	emb_matrix = np.array(emb_matrix)
	for i in range(emb_size):
		tmp['{}_{}_graph_emb_{}'.format(f1, f2, i)] = emb_matrix[:, i]
	
	word_list = []
	emb_matrix2 = []
	for w in model.wv.vocab:
		word_list.append(w)
		emb_matrix2.append(model.wv[w])
	emb_matrix2 = np.array(emb_matrix2)
	tmp2 = pd.DataFrame()
	tmp2[f2 + '_word'] = word_list
	for i in range(8):
		tmp2['{}_{}_graph_emb_{}'.format(f2, f1, i)] = emb_matrix2[:, i]
	return tmp, tmp2


**针对于test数据集的数据分析**

In [ ]:
test_ads["time_YMD"]=test_ads["pt_d"]//10000-20220603
test_ads["hour"]=test_ads["pt_d"]%10000//100

In [ ]:
test_ads["time_YMD"].value_counts()

In [ ]:
import seaborn  as sns
ff=[]
def field_plot():
    for i in train_ads.columns:
        if train_ads[i].nunique() < 20:
            ff.append(i)
    for f in ff:
        print(f'Distribution of {f} :', train_ads[f].nunique())
        print(train_ads[f].value_counts()/len(train_ads),'\n')
        # plt.figure(figsize=(15,10))
        # sns.countplot(x=f,hue='label',data=train_ads)

field_plot()   

In [ ]:
import seaborn  as sns
ff=[]
def field_plot():
    for i in test_ads.columns:
        if test_ads[i].nunique() < 20:
            ff.append(i)
    for f in ff:
        print(f'Distribution of {f} :', test_ads[f].nunique())
        print(test_ads[f].value_counts()/len(test_ads),'\n')
        # plt.figure(figsize=(15,10))
        # sns.countplot(x=f,hue='label',data=train_ads)

field_plot()   

In [ ]:
train_ads.drop("label",axis=1,inplace=True)

In [ ]:
full_data=pd.concat([train_ads,test_ads],ignore_index=True)

In [ ]:
train_ads

In [ ]:
test_ads

In [ ]:
full_data["time_YMD"].value_counts()

In [ ]:
full_data["user_id"].nunique()

In [ ]:
cold_num=[]

for i in range(1,8):
    uid_list=full_data[full_data["time_YMD"]<i-1]["user_id"].unique()
    cold_list=[]
    old_list=[]
    k=0
    for j  in full_data[full_data["time_YMD"]==i]["user_id"].unique():
        if j not in uid_list:
            k+=1
            cold_list.append(j)
        else:
            old_list.append(j)
    cold_num.append(k)

In [ ]:
cold_num

In [ ]:
full_data.groupby(["user_id"])["log_id"].transform('count').sort_values(ascending=False)

In [ ]:
len(cold_list)

In [ ]:
len(old_list)

In [ ]:
df=full_data[full_data["time_YMD"]==7]

In [ ]:
df

In [ ]:
i=0
j=0
k=0
a=0
b=0
c=0

In [ ]:
for t1,t2 in tqdm(df.iterrows()):
    if t2[1] in cold_list:     
        i+=1
        
    else:
        a+=1
       

In [ ]:
i

In [ ]:
a

In [ ]:
i+a

In [ ]:
i/1932

In [ ]:
a/27003

In [ ]:
full_data.info()

In [ ]:
full_data.groupby('user_id')['log_id'].count().max() #最多的个人被推荐过4173次，最少为两次

In [ ]:
#用户重复点击
user_click_count = full_data.groupby(['user_id', 'task_id'])['log_id'].agg({'count'}).reset_index()
user_click_count.sort_values(by="count",ascending=False)[:20]

In [ ]:
full_data.columns

In [ ]:
user_click_item_count = sorted(full_data.groupby('user_id')['task_id'].count(), reverse=True)
plt.plot(user_click_item_count)

In [ ]:
plt.plot(user_click_item_count[:200])

In [ ]:
user_click_item_count[10000]

In [ ]:
def plot_envs(df, cols, r, c):
    plt.figure()
    plt.figure(figsize=(10, 5))
    i = 1
    for col in cols:
        plt.subplot(r, c, i)
        i += 1
        v = df[col].value_counts().reset_index()
        fig = sns.barplot(x=v['index'], y=v[col])
        for item in fig.get_xticklabels():
            item.set_rotation(90)
        plt.title(col)
    plt.tight_layout()
    plt.show()

In [ ]:
# 分析用户点击环境变化是否明显，这里随机采样10个用户分析这些用户的点击环境分布
sample_user_ids = np.random.choice(full_data['user_id'].unique(), size=10, replace=False)
sample_users = full_data[full_data['user_id'].isin(sample_user_ids)]
cols = ['gender', 'residence', 'city', 'city_rank',
       'series_dev', 'series_group', 'emui_dev', 'device_name', 'device_size',
       'net_type']
for _, user_df in sample_users.groupby('user_id'):
    plot_envs(user_df, cols, 2, 5)

In [ ]:
sample_user_ids

In [ ]:
full_data[full_data["user_id"]==267202]["device_size"].value_counts()

In [ ]:
item_click_count = sorted(full_data.groupby('task_id')['user_id'].count(), reverse=True)
plt.plot(item_click_count[:200])

In [ ]:
item_click_count

In [ ]:
full_data.columns

In [ ]:
tmp = full_data.sort_values('pt_d')
tmp['next_item'] = tmp.groupby(['user_id'])['task_id'].transform(lambda x:x.shift(-1))
union_item = tmp.groupby(['task_id','next_item'])['pt_d'].agg({'count'}).reset_index().sort_values('count', ascending=False)
union_item[['count']].describe()

In [ ]:
x = union_item['task_id']
y = union_item['count']
plt.scatter(x, y)

In [ ]:
union_item

In [ ]:
plt.plot(union_item['count'].values[620000:]) #25万个

In [ ]:
full_data["creat_type_cd"].value_counts()

In [ ]:
full_data

In [ ]:
plt.plot(sorted(full_data.groupby('user_id')['creat_type_cd'].nunique(), reverse=True))

**双塔模型**

In [ ]:
full_data.columns

***特征工程***

In [ ]:
def active_level(all_data, cols):
    """
    制作区分用户活跃度的特征
    :param all_data: 数据集
    :param cols: 用到的特征列
    """
    data = all_data[cols]
    data.sort_values(['user_id', 'pt_d'], inplace=True)
    user_act = pd.DataFrame(data.groupby('user_id', as_index=False)[['task_id', 'pt_d']].\
                            agg({'task_id':np.size, 'pt_d': {list}}).values, columns=['user_id', 'click_size', 'pt_d'])
    
    # 计算时间间隔的均值
    def time_diff_mean(l):
        if len(l) == 1:
            return 1
        else:
            return np.mean([ minite(j)-minite(i) for i, j in list(zip(l[:-1], l[1:]))])
    
    def minite(j):
        return ((j%1000000)//10000)*24*60+((j%100000000))//100*60+(j%100)

    user_act['time_diff_mean'] = user_act['pt_d'].apply(lambda x: time_diff_mean(x))
    
    # 点击次数取倒数
    user_act['click_size'] = 1 / user_act['click_size']
    
    # 两者归一化
    user_act['click_size'] = (user_act['click_size'] - user_act['click_size'].min()) / (user_act['click_size'].max() - user_act['click_size'].min())
    user_act['time_diff_mean'] = (user_act['time_diff_mean'] - user_act['time_diff_mean'].min()) / (user_act['time_diff_mean'].max() - user_act['time_diff_mean'].min())     
    user_act['active_level'] = user_act['click_size'] + user_act['time_diff_mean']
    
    user_act['user_id'] = user_act['user_id'].astype('int')
    del user_act['pt_d']
    
    return user_act

user_act_fea = active_level(full_data, ['user_id', 'task_id', 'pt_d'])

In [ ]:
user_act_fea.head()

In [ ]:
def hot_level(all_data, cols):
    """
    制作衡量文章热度的特征
    :param all_data: 数据集
    :param cols: 用到的特征列
    """
    data = all_data[cols]
    data.sort_values(['task_id', 'pt_d'], inplace=True)
    article_hot = pd.DataFrame(data.groupby('task_id', as_index=False)[['user_id', 'pt_d']].\
                               agg({'user_id':np.size, 'pt_d': {list}}).values, columns=['task_id', 'user_num', 'pt_d'])
    
    # 计算被点击时间间隔的均值
    def time_diff_mean(l):
        if len(l) == 1:
            return 1
        else:
            return np.mean([ minite(j)-minite(i) for i, j in list(zip(l[:-1], l[1:]))])
    
    def minite(j):
        return ((j%1000000)//10000)*24*60+((j%100000000))//100*60+(j%100)

    
    # 点击次数取倒数
    article_hot['user_num'] = 1 / article_hot['user_num']
    article_hot['time_diff_mean'] = article_hot['pt_d'].apply(lambda x: time_diff_mean(x))
    # 两者归一化
    article_hot['user_num'] = (article_hot['user_num'] - article_hot['user_num'].min()) / (article_hot['user_num'].max() - article_hot['user_num'].min())
    article_hot['time_diff_mean'] = (article_hot['time_diff_mean'] - article_hot['time_diff_mean'].min()) / (article_hot['time_diff_mean'].max() - article_hot['time_diff_mean'].min())     
    article_hot['hot_level'] = article_hot['user_num'] + article_hot['time_diff_mean']
    
    #article_hot['click_article_id'] = article_hot['click_article_id'].astype('int')
    
    del article_hot['pt_d']
    
    return article_hot

In [ ]:
article_hot_fea = hot_level(full_data, ['user_id', 'task_id', 'pt_d'])    

In [ ]:
article_hot_fea.head()

In [ ]:
def user_time_hob_fea(all_data, cols):
    """
    制作用户的时间习惯特征
    :param all_data: 数据集
    :param cols: 用到的特征列
    """
    user_time_hob_info = all_data[cols]

    user_time_hob_info = user_time_hob_info.groupby('user_id').agg('mean').reset_index()
    
    user_time_hob_info.rename(columns={'hour': 'hour_hob1', 'time_YMD': 'time_YMD_hob1'}, inplace=True)
    return user_time_hob_info

In [ ]:
user_time_hob_cols = ['user_id', 'hour', 'time_YMD']
user_time_hob_info = user_time_hob_fea(full_data, user_time_hob_cols)

In [ ]:
user_time_hob_info

In [ ]:
def user_cat_hob_fea(all_data, cols):
    """
    用户的主题爱好
    :param all_data: 数据集
    :param cols: 用到的特征列
    """
    user_category_hob_info = all_data[cols]
    user_category_hob_info = user_category_hob_info.groupby('user_id').agg({list}).reset_index()
    
    user_cat_hob_info = pd.DataFrame()
    user_cat_hob_info['user_id'] = user_category_hob_info['user_id']
    user_cat_hob_info['cate_list'] = user_category_hob_info['creat_type_cd']
    
    return user_cat_hob_info

In [ ]:
user_category_hob_cols = ['user_id', 'creat_type_cd']
user_cat_hob_info = user_cat_hob_fea(full_data, user_category_hob_cols)

In [ ]:
user_cat_hob_info.info

In [ ]:
user_info = user_act_fea.merge(user_time_hob_info, on='user_id')
user_info = user_info.merge(user_cat_hob_info, on='user_id')
full_data=full_data.merge(user_info,on="user_id",how="left")
full_data=full_data.merge(article_hot_fea,on="task_id",how="left")

In [ ]:
full_data

***YOUTubeDNN***

In [2]:
import numpy

In [44]:
import pandas as pd
train_ads = pd.read_pickle('d:/Data/2022_3_data/train/train_data_ads.pkl')

In [4]:
import pandas as pd  
import numpy as np
from tqdm import tqdm  
from collections import defaultdict  
import os, math, warnings, math, pickle
from tqdm import tqdm
import faiss
import collections
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from deepctr.feature_column import SparseFeat, VarLenSparseFeat,DenseFeat
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from deepmatch.models import *
from deepmatch.utils import sampledsoftmaxloss
from sklearn.metrics import log_loss, roc_auc_score
warnings.filterwarnings('ignore')
from deepmatch.utils import sampledsoftmaxloss, NegativeSampler


In [5]:
train_ads.columns

Index(['log_id', 'label', 'user_id', 'age', 'gender', 'residence', 'city',
       'city_rank', 'series_dev', 'series_group', 'emui_dev', 'device_name',
       'device_size', 'net_type', 'task_id', 'adv_id', 'creat_type_cd',
       'adv_prim_id', 'inter_type_cd', 'slot_id', 'site_id', 'spread_app_id',
       'hispace_app_tags', 'app_second_class', 'app_score',
       'ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003',
       'ad_close_list_v001', 'ad_close_list_v002', 'ad_close_list_v003',
       'pt_d', 'u_newsCatInterestsST', 'u_refreshTimes', 'u_feedLifeCycle'],
      dtype='object')

In [6]:
cols_dict = {'user': ['user id','age','gender' , 'residence' , 'city', 'city_rank', 'series dev', 'series group', 'emui dev','device name','device size', 'net_type'],
'item': ['task id', 'adv id','creat_type_cd','adv_prim id', 'inter type cd','slot id', 'spread app id','app second class','log_id'],
'cate':['hour', 'task id','adv_id', 'creat_type_cd', 'user_ id','age', 'gender','residence','city rank', 'series dev','series group','emui dev','device name','device size','city','net type','inter type cd','slot_id', 'spread_app_id', 'app_second_class' ]}

In [7]:
def youtubednn_recall(data,topk=200, embedding_dim=16, his_seq_maxlen=50, negsample=0,
                      batch_size=64, epochs=1, verbose=1, validation_split=0.0):
    """通过YouTubeDNN模型，计算用户向量和文章向量
    param: data: 用户日志数据
    topk: 对于每个用户，召回多少篇文章
    """
    user_id_raw = data[['user_id']].drop_duplicates('user_id',keep="last")
    doc_id_raw = data[['adv_id']].drop_duplicates('adv_id',keep="last")
    
    # 类别数据编码   
    cate=['task_id','adv_id', 'creat_type_cd', 'user_id','age', 'gender',
    'residence','city_rank', 'series_dev','series_group','emui_dev','device_name',
    'device_size','city','net_type','inter_type_cd','slot_id', 'spread_app_id', 'app_second_class' ]
    feature_max_idx = {}
    for f in cate:
        lbe = LabelEncoder()
        data[f] = lbe.fit_transform(data[f])
        feature_max_idx[f] = data[f].max() + 1
        
    # 构建用户id词典和doc的id词典，方便从用户idx找到原始的id
    user_id_enc = data[['user_id']].drop_duplicates('user_id',keep="last")
    doc_id_enc = data[['adv_id']].drop_duplicates('adv_id',keep="last")
    user_idx_2_rawid = dict(zip(user_id_enc['user_id'], user_id_raw['user_id']))
    doc_idx_2_rawid = dict(zip(doc_id_enc['adv_id'], doc_id_raw['adv_id']))
    
    # 保存下每篇文章的被点击数量， 方便后面高热文章的打压
    doc_clicked_count_df = data.groupby('adv_id')['log_id'].apply(lambda x: x.count()).reset_index()
    doc_clicked_count_dict = dict(zip(doc_clicked_count_df['adv_id'], doc_clicked_count_df['log_id']))

    train_set, test_set = gen_data_set(data, doc_clicked_count_dict, negsample, control_users=True)
    
    # full_data=train_set+test_set
    # random.shuffle(full_data)
    # # 构造youtubeDNN模型的输入
    # train_model_input, train_label = gen_model_input(train_set, his_seq_maxlen)
    # test_model_input, test_label = gen_model_input(test_set, his_seq_maxlen)
    
    # full_data_input,full_data_label=gen_model_input(full_data,his_seq_maxlen)
    # # 构建模型并完成训练
    # model = train_youtube_model(train_model_input, train_label, embedding_dim, feature_max_idx, his_seq_maxlen, batch_size, epochs, verbose, validation_split)
    
    # # 获得用户embedding和doc的embedding， 并进行保存
    # user_embs, doc_embs = get_embeddings(model, full_data_input, user_idx_2_rawid, doc_idx_2_rawid)
    
    # # 对每个用户，拿到召回结果并返回回来
    # #user_recall_doc_dict = get_youtube_recall_res(user_embs, doc_embs, user_idx_2_rawid, doc_idx_2_rawid, topk)
    
    return train_set,test_set


In [31]:
train_set,test_set=youtubednn_recall(train_ads)

100%|██████████| 65297/65297 [02:37<00:00, 414.60it/s] 


KeyboardInterrupt: 

In [ ]:
train_set

In [8]:
 #获取用户embedding和文章embedding
def get_embeddings(model, test_model_input, user_idx_2_rawid, doc_idx_2_rawid, save_path='d:/Data/2022_3_data/train/'):
    doc_model_input = {'adv_id':np.array(list(doc_idx_2_rawid.keys()))}
    
    user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
    doc_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)
    
    # 保存当前的item_embedding 和 user_embedding 排序的时候可能能够用到，但是需要注意保存的时候需要和原始的id对应
    user_embs = user_embedding_model.predict(test_model_input, batch_size=2 ** 12)
    doc_embs = doc_embedding_model.predict(doc_model_input, batch_size=2 ** 12)
    # embedding保存之前归一化一下
    user_embs = user_embs / np.linalg.norm(user_embs, axis=1, keepdims=True)
    doc_embs = doc_embs / np.linalg.norm(doc_embs, axis=1, keepdims=True)
    
    # 将Embedding转换成字典的形式方便查询
    raw_user_id_emb_dict = {user_idx_2_rawid[k]: \
                                v for k, v in zip(user_idx_2_rawid.keys(), user_embs)}
    raw_doc_id_emb_dict = {doc_idx_2_rawid[k]: \
                                v for k, v in zip(doc_idx_2_rawid.keys(), doc_embs)}
    # 将Embedding保存到本地
    pickle.dump(raw_user_id_emb_dict, open(save_path + 'user_youtube_emb.pkl', 'wb'))
    pickle.dump(raw_doc_id_emb_dict, open(save_path + 'doc_youtube_emb.pkl', 'wb'))
    
    # 读取
    #user_embs_dict = pickle.load(open('embedding/user_youtube_emb.pkl', 'rb'))
    #doc_embs_dict = pickle.load(open('embedding/doc_youtube_emb.pkl', 'rb'))
    return raw_user_id_emb_dict, raw_doc_id_emb_dict


In [9]:
train_ads["pt_d"]

0          202206030326
1          202206030326
2          202206030326
3          202206030326
4          202206030328
               ...     
7675512    202206090848
7675513    202206090848
7675514    202206091012
7675515    202206091119
7675516    202206090846
Name: pt_d, Length: 7675517, dtype: int64

In [10]:
user=['age','gender' , 'residence' , 'city', 'city_rank', 'series dev', 'series group', 'emui dev','device name','device size', 'net_type']      
item=['task id', 'adv id','creat_type_cd','adv_prim id', 'inter type cd','slot id', 'spread app id','app second class','log_id']
def gen_data_set(click_data, doc_clicked_count_dict, negsample, control_users=False):
    """构造youtubeDNN的数据集"""
    # 按照曝光时间排序
    click_data.sort_values("pt_d", inplace=True)
    item_ids = click_data['adv_id'].unique()
    
    train_set, test_set = [], []
    for user_id, hist_click in tqdm(click_data.groupby('user_id')):
        # 这里按照expo_date分开，每一天用滑动窗口滑，可能相关性更高些,另外，这样序列不会太长，因为eda发现有点击1111个的
        #for expo_date, hist_click in hist_date_click.groupby('expo_date'):
        # 用户当天的点击历史id
        pos_list = hist_click['adv_id'].tolist()
        label_list=hist_click['label'].tolist()
        pt_d_list=hist_click["pt_d"].tolist()
        age_list=hist_click["age"].tolist()
        gender_list=hist_click["gender"].tolist()
        residence_list=hist_click["residence"].tolist()
        city_list=hist_click["city"].tolist()
        city_rank_list=hist_click["city_rank"].tolist()
        series_dev_list=hist_click['series_dev'].tolist()
        series_group_list=hist_click['series_group'].tolist()
        emui_dev_list=hist_click['emui_dev'].tolist()
        device_name_list=hist_click['device_name'].tolist()
        device_size_list=hist_click['device_size'].tolist()
        net_type_list=hist_click['net_type'].tolist() 
   
        #user_control_flag = True
        
        if control_users:
            user_samples_cou = 0
        
        # 过长的序列截断
       
        #if negsample > 0:
        #    neg_list = gen_neg_sample_candiate(pos_list, item_ids, doc_clicked_count_dict, negsample, methods='multinomial')
        
        # 只有1个的也截断 去掉，当然我之前做了处理，这里没有这种情况了
        if len(pos_list) < 1:
            continue
        else: 
            for i in range(0, len(pos_list)):
                hist = pos_list[:i]
                row = [user_id, hist[::-1], pos_list[i],age_list[i],gender_list[i],residence_list[i],city_list[i]
                ,city_rank_list[i],series_dev_list[i],series_group_list[i],emui_dev_list[i],device_name_list[i], 
                device_size_list[i],net_type_list[i],label_list[i], len(hist[::-1])]
                if pt_d_list[i]<202206070000:
                    train_set.append(row)
                else:
                    test_set.append(row)
              
    random.shuffle(train_set)
    random.shuffle(test_set)
    
    return train_set, test_set   


In [11]:
def gen_model_input(train_set, his_seq_max_len):
    """构造模型的输入"""
    # row: [user_id, hist_list, cur_doc_id, city, age, gender, label, hist_len]
    train_uid = np.array([row[0] for row in train_set])
    train_hist_seq = [row[1] for row in train_set]
    train_iid = np.array([row[2] for row in train_set])
    train_u_age = np.array([row[3] for row in train_set])
    train_u_gender = np.array([row[4] for row in train_set])
    train_u_residence=np.array([row[5] for row in train_set])
    train_u_city = np.array([row[6] for row in train_set])
    train_u_city_rank = np.array([row[7] for row in train_set])
    train_u_series_dev=np.array([row[8] for row in train_set])
    train_u_series_group=np.array([row[9] for row in train_set])
    train_u_emui_dev=np.array([row[10] for row in train_set])
    train_u_device_name = np.array([row[11] for row in train_set])
    train_u_device_size = np.array([row[12] for row in train_set])
    train_u_net_type_list=np.array([row[13] for row in train_set])
    train_label = np.array([row[14] for row in train_set])
    train_hist_len = np.array([row[15] for row in train_set])
    train_seq_pad = pad_sequences(train_hist_seq, maxlen=his_seq_max_len, padding='post', truncating='post', value=0)
    train_model_input = {
        "user_id": train_uid,
        "adv_id": train_iid,
        "hist_doc_ids": train_seq_pad,
        "hist_len": train_hist_len,
        "u_city": train_u_city,
        "u_age": train_u_age,
        "u_gender": train_u_gender,
        "residence": train_u_residence,
        "city_rank":train_u_city_rank,
        "series_dev":train_u_series_dev,
        "series_group":train_u_series_group,
        "emui_dev":train_u_emui_dev,
        "device_name":train_u_device_name,
        "device_size":train_u_device_size,
        "net_type_list":train_u_net_type_list
    }
    return train_model_input, train_label


In [13]:
user=['age','gender' , 'residence' , 'city', 'city_rank', 'series dev',
 'series group', 'emui dev','device name','device size', 'net_type']      
def train_youtube_model(train_model_input, train_label, embedding_dim, feature_max_idx, his_seq_maxlen, batch_size, epochs, verbose, validation_split):
    """构建youtubednn并完成训练"""
    # 特征封装
    user_feature_columns = [
        SparseFeat('user_id', feature_max_idx['user_id'], embedding_dim),
        VarLenSparseFeat(SparseFeat('hist_doc_ids', feature_max_idx['adv_id'], embedding_dim,
                                                        embedding_name="adv_id"), his_seq_maxlen, 'mean', 'hist_len'),    
        
        SparseFeat('u_city', feature_max_idx['city'], embedding_dim),
        SparseFeat('u_age', feature_max_idx['age'], embedding_dim),
        SparseFeat('u_gender', feature_max_idx['gender'], embedding_dim),
        SparseFeat('residence',feature_max_idx["residence"],embedding_dim),
        SparseFeat('city_rank',feature_max_idx["city_rank"],embedding_dim),
        SparseFeat('series_dev',feature_max_idx["series_dev"],embedding_dim),
        SparseFeat('series_group',feature_max_idx["series_group"],embedding_dim),
        SparseFeat('emui_dev',feature_max_idx["emui_dev"],embedding_dim),
        SparseFeat('device_name',feature_max_idx["device_name"],embedding_dim),
        SparseFeat('device_size',feature_max_idx["device_size"],embedding_dim),
        SparseFeat('net_type_list',feature_max_idx["net_type"],embedding_dim),
        DenseFeat('hist_len', 1,)
    ]
    doc_feature_columns = [
        SparseFeat('adv_id', feature_max_idx['adv_id'], embedding_dim)
        # 这里后面也可以把文章的类别画像特征加入
    ]

    
    
    from collections import Counter
    train_counter = Counter(train_ads['adv_id'])
    item_count = [train_counter.get(i, 0) for i in range(doc_feature_columns[0].vocabulary_size)]
    sampler_config = NegativeSampler('frequency', num_sampled=5, item_name='adv_id', item_count=item_count)
    import tensorflow as tf
    if tf.__version__ >= '2.0.0':
        tf.compat.v1.disable_eager_execution()
    else:
        K.set_learning_phase(True)

    # 定义模型
    model = YoutubeDNN(user_feature_columns, doc_feature_columns, sampler_config=sampler_config,user_dnn_hidden_units=(64, embedding_dim))
    
    # 模型编译
    model.compile(optimizer="adam", loss=sampledsoftmaxloss)
    
    # 模型训练，这里可以定义验证集的比例，如果设置为0的话就是全量数据直接进行训练
    history = model.fit(train_model_input, train_label, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_split=validation_split)
    
    return model


In [14]:
user_embs,doc_embs=youtubednn_recall(train_ads)

100%|██████████| 65297/65297 [01:12<00:00, 894.79it/s] 


Train on 4482622 samples
4482622/4482622 [==============================] - 1046s 233us/sample - loss: 1.3437
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


In [38]:
user

[100005,
 100157,
 100345,
 100381,
 100583,
 100658,
 100803,
 101021,
 101273,
 101302,
 101326,
 101371,
 101676,
 101834,
 101851,
 101872,
 101902,
 102013,
 102016,
 102034,
 102093,
 102119,
 102498,
 102517,
 102553,
 102570,
 102663,
 102712,
 102829,
 102871,
 103031,
 103071,
 103140,
 103170,
 103196,
 103382,
 103559,
 103623,
 103739,
 103764,
 103823,
 103834,
 103864,
 103972,
 103976,
 104082,
 104204,
 104207,
 104221,
 104255,
 104259,
 104518,
 104617,
 104773,
 104887,
 104941,
 105006,
 105051,
 105238,
 105253,
 105295,
 105336,
 105337,
 105351,
 105445,
 105476,
 105687,
 105716,
 105790,
 105833,
 106048,
 106214,
 106235,
 106249,
 106284,
 106431,
 106681,
 106761,
 106806,
 106820,
 106822,
 106922,
 106925,
 106971,
 107050,
 107059,
 107078,
 107099,
 107121,
 107326,
 107461,
 107506,
 107578,
 107709,
 107732,
 107766,
 107915,
 108137,
 108206,
 108414,
 108431,
 108672,
 108726,
 108728,
 108828,
 108854,
 108932,
 108947,
 108979,
 109130,
 109172,
 

In [43]:
sorted(list(train_ads["user_id"].unique()))

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [45]:
k=0
for i in train_ads["user_id"].unique():
    if i not in user:
        k+=1

In [46]:
k

0

In [ ]:
len(user)

65297

In [16]:
user=pd.read_pickle("D:/Data/2022_3_data/train/user_youtube_emb.pkl")

In [17]:
user

{100005: array([-0.16797973,  0.00610521, -0.11816724,  0.20495535, -0.3682848 ,
         0.12736481,  0.46596056,  0.3195587 ,  0.04699069, -0.24701428,
        -0.03725084, -0.44443595, -0.1070642 , -0.36003166,  0.00303326,
        -0.20359997], dtype=float32),
 100157: array([-0.04395837, -0.64995867,  0.09959335,  0.10113312, -0.0184658 ,
         0.09973828,  0.08302409,  0.1205925 ,  0.27033773,  0.32758504,
         0.04875058, -0.20719098, -0.05203911,  0.22466525,  0.20238368,
        -0.4515634 ], dtype=float32),
 100345: array([-0.15675718, -0.00867519, -0.04517262,  0.4110094 , -0.2961253 ,
        -0.04469588,  0.2824834 ,  0.38164887,  0.102948  , -0.06130155,
         0.00418286, -0.11013386, -0.09343119, -0.6272899 , -0.01281274,
        -0.24566787], dtype=float32),
 100381: array([-0.6057349 ,  0.0722781 , -0.22279882,  0.2544319 , -0.22602576,
        -0.02784285,  0.3094882 ,  0.22595961,  0.07648965, -0.17505081,
         0.00706758,  0.22469798, -0.15990445, -0.2

***deepFM***

In [ ]:
train_ads.columns

In [ ]:
np.bincount(train_ads["label"])

In [ ]:
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, DenseFeat,get_feature_names

if __name__ == "__main__":
    data = train_ads

    dense_features= ['hour', 'task_id','adv_id', 'creat_type_cd', 'user_id','age', 'gender','residence','city_rank', 'series_dev','series_group','emui_dev','device_name','device_size','city','net_type','inter_type_cd','slot_id', 'spread_app_id', 'app_second_class' ]
    sparse_features = [item for item in list(train_ads) if item not in sparse_features]

    data[sparse_features] = data[sparse_features].fillna('-1', )
    data[dense_features] = data[dense_features].fillna(0, )
    target = ['label']

    # 1.do simple Transformation for dense features
    # mms = MinMaxScaler(feature_range=(0, 1))
    # data[dense_features] = mms.fit_transform(data[dense_features])

    # 2.set hashing space for each sparse field,and record dense feature field name

    fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(),embedding_dim="auto", use_hash=True, dtype='string')  # since the input is string
                              for feat in sparse_features] + [DenseFeat(feat, 1, ) for feat in dense_features]
     
    linear_feature_columns = fixlen_feature_columns
    dnn_feature_columns = fixlen_feature_columns
    feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns, )

    # 3.generate input data for model

    train, test = data[data["time_YMD"]<5],data[data["time_YMD"]>=5] 

    train_model_input = {name:train[name] for name in feature_names}
    test_model_input = {name:test[name] for name in feature_names}


    # 4.Define Model,train,predict and evaluate
    model = DeepFM(linear_feature_columns,dnn_feature_columns, task='binary',dnn_dropout=0.2,l2_reg_dnn=0.5,dnn_hidden_units=(1024,256, 128, 64))
    model.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'], )

    history = model.fit(train_model_input, train[target].values,batch_size=512, epochs=10, verbose=2, validation_split=0.2,class_weight={0:0.2,1:9.8})
    pred_ans = model.predict(test_model_input, batch_size=256)
    print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
    print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))


***SDM***

In [1]:
import pandas as pd
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from preprocess import gen_data_set, gen_model_input,gen_data_set_sdm,gen_model_input_sdm
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model

from deepmatch.models import *
from deepmatch.utils import sampledsoftmaxloss, NegativeSampler

In [ ]:
data_path = "./"

unames = ['user_id','gender','age','occupation','zip']
user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames)
rnames = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames)
mnames = ['movie_id','title','genres']
movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=mnames,encoding="unicode_escape")
movies['genres'] = list(map(lambda x: x.split('|')[0], movies['genres'].values))

data = pd.merge(pd.merge(ratings,movies),user)#.iloc[:10000]

In [ ]:
#data = pd.read_csvdata = pd.read_csv("./movielens_sample.txt")
sparse_features = ['user_id','movie_id','gender', 'age', 'occupation', 'zip', 'genres']
SEQ_LEN = 50
SEQ_LEN_short = 5
SEQ_LEN_prefer = 50
negsample = 0

# 1.Label Encoding for sparse features,and process sequence features with `gen_date_set` and `gen_model_input`

feature_max_idx = {}
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1
    feature_max_idx[feature] = data[feature].max() + 1
features = sparse_features
user_profile = data[features].drop_duplicates('user_id')

item_profile = data[["movie_id"]].drop_duplicates('movie_id')

user_profile.set_index("user_id", inplace=True)

user_item_list = data.groupby("user_id")['movie_id'].apply(list)

train_set, test_set = gen_data_set_sdm(data, seq_short_max_len=SEQ_LEN_short, seq_prefer_max_len=SEQ_LEN_prefer)

train_model_input, train_label = gen_model_input_sdm(train_set, user_profile, SEQ_LEN_short, SEQ_LEN_prefer)
test_model_input, test_label = gen_model_input_sdm(test_set, user_profile, SEQ_LEN_short, SEQ_LEN_prefer)

# 2.count #unique features for each sparse field and generate feature config for sequence feature

embedding_dim = 32


user_feature_columns = [SparseFeat('user_id', feature_max_idx['user_id'], 16),
                        SparseFeat("gender", feature_max_idx['gender'], 16),
                        SparseFeat("age", feature_max_idx['age'], 16),
                        SparseFeat("occupation", feature_max_idx['occupation'], 16),
                        SparseFeat("zip", feature_max_idx['zip'], 16),
                        VarLenSparseFeat(SparseFeat('short_movie_id', feature_max_idx['movie_id'], embedding_dim,
                                                    embedding_name="movie_id"), SEQ_LEN_short, 'mean',
                                         'short_sess_length'),
                        VarLenSparseFeat(SparseFeat('prefer_movie_id', feature_max_idx['movie_id'], embedding_dim,
                                                    embedding_name="movie_id"), SEQ_LEN_prefer, 'mean',
                                         'prefer_sess_length'),
                        VarLenSparseFeat(SparseFeat('short_genres', feature_max_idx['genres'], embedding_dim,
                                                    embedding_name="genres"), SEQ_LEN_short, 'mean',
                                         'short_sess_length'),
                        VarLenSparseFeat(SparseFeat('prefer_genres', feature_max_idx['genres'], embedding_dim,
                                                    embedding_name="genres"), SEQ_LEN_prefer, 'mean',
                                         'prefer_sess_length'),
                        ]


item_feature_columns = [SparseFeat('movie_id', feature_max_idx['movie_id'], embedding_dim)]

from collections import Counter
train_counter = Counter(train_model_input['movie_id'])
item_count = [train_counter.get(i,0) for i in range(item_feature_columns[0].vocabulary_size)]
sampler_config = NegativeSampler('frequency',num_sampled=255,item_name="movie_id",item_count=item_count)

# 3.Define Model and train

import tensorflow as tf
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()
else:
    K.set_learning_phase(True)

model = SDM(user_feature_columns, item_feature_columns, history_feature_list=['movie_id','genres'],
            units=embedding_dim, sampler_config=sampler_config )

model.compile(optimizer="adam", loss=sampledsoftmaxloss)

history = model.fit(train_model_input, train_label,  # train_label,
                    batch_size=512, epochs=20, verbose=1, validation_split=0.0, )

# 4. Generate user features for testing and full item features for retrieval
test_user_model_input = test_model_input
all_item_model_input = {"movie_id": item_profile['movie_id'].values,}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_embs = user_embs[:, i, :]  # i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)

In [ ]:
test_true_label = {line[0]:[line[1]] for line in test_set}

import numpy as np
import faiss
from tqdm import tqdm
from deepmatch.utils import recall_N

index = faiss.IndexFlatIP(embedding_dim)
# faiss.normalize_L2(item_embs)
index.add(item_embs)
# faiss.normalize_L2(user_embs)
D, I = index.search(np.ascontiguousarray(user_embs), 50)
s = []
hit = 0
for i, uid in tqdm(enumerate(test_user_model_input['user_id'])):
    try:
        pred = [item_profile['movie_id'].values[x] for x in I[i]]
        filter_item = None
        recall_score = recall_N(test_true_label[uid], pred, N=50)
        s.append(recall_score)
        if test_true_label[uid] in pred:
            hit += 1
    except:
        print(i)
print("")
print("recall", np.mean(s))
print("hit rate", hit / len(test_user_model_input['user_id']))